# Neo4j modul telepítése és csatlakozás az adatbázishoz

In [1]:
# Telepítése
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=23d06807b1edca5fc931a86b4039743d65ffbb3db11fb022a4edcabfb2277aad
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
Successfully built neo4j


In [2]:
# Importálás
from neo4j import GraphDatabase, Driver, Session, Result, basic_auth

## Csatlakozás megvalósítása osztállyal

- Neo4j instance elérése
- Autentikáció
- Cypyher query futtatása

In [3]:
# Connection osztály
class Neo4jConnection:
  # Dunder methods
  def __init__(self, uri: str, user: str, password: str):
    """
    Instantiate a new Neo4jConnection objection with the specified values and try to establish connection.

    :param str uri: The connection URI used for the driver.
    :param str user: The username used for atuhenticating the connection.
    :param str password: The password used for authenticating the connection.
    """
    # Create attributes with default values and type hints
    self.__uri: str = ''
    self.__user: str = ''
    self.__password: str = ''
    self.__driver = None

    # Create driver and connect to instance
    self.connect(uri, user, password)

  def __del__(self):
    """
    Close existing connections before destroying the Neo4jCOnnection object.
    """
    self.close()

  # Getter and setter functions
  def __getUri(self):
    return self.__uri

  def __getUser(self):
    return self.__user

  def __getPassword(self):
    return self.__password

  def __getAuth(self):
    return (self.__getUser(), self.__getPassword())

  def __setUri(self, value: str):
    self.__uri = value

  def __setUser(self, value: str):
    self.__user = value

  def __setPassword(self, value: str):
    self.__password = value

  # Properties
  URI: str = property(__getUri, __setUri)
  """
  Unified Resource Identifier used for driver connection.
  """
  User: str = property(__getUser, __setUser)
  """
  The username used for authentication.
  """
  Password: str = property(__getPassword, __setPassword)
  """
  The password used for authentication.
  """
  Auth: tuple[str, str] = property(__getAuth)
  """
  Authentication details (username and password).
  """


  def connect(self, uri: str = ..., user: str = ..., password: str = ...):
    """
    Create a Neo4j driver and connect to an instance.

    :param str uri: The connection URI used for the driver.
    :param str user: The username used for atuhenticating the connection.
    :param str password: The password used for authenticating the connection.
    """
    # Check if any parameters were passed
    self.URI = (uri if uri is not Ellipsis else self.URI)
    self.User = (user if user is not Ellipsis else self.User)
    self.Password = (password if password is not Ellipsis else self.Password)

    try:
      # Establish connection and authenticate
      self.__driver: Driver = GraphDatabase.driver(
          self.URI,
          auth=basic_auth(self.User, self.Password)
      )

    except Exception as e:
      # Catch any errors
      print("Failed to establish connection.\n", e)

  def close(self):
    """
    Close the driver's connection to the instance.
    """
    if self.__driver is not None: self.__driver.close()

  def run(self, query:str, database: str = ..., **kwargs):
    """
    Run a Cypher query on the estalbished connetion.
    """
    # Declare variables with type hints
    session: Session = None
    result: list = []

    try:
      # Check if the driver is working as intended
      assert self.__driver is not None, "Driver is not instantiated!"
      self.__driver.verify_connectivity()
      self.__driver.verify_authentication()

      session = self.__driver.session(database=database) if database is not Ellipsis else self.__driver.session()
      print(f"Running query: \"{query}\"")
      result = list(session.run(query))

    except Exception as e:
      # Catch any errors
      print("An exception occured when trying to communicate with the server.\n", e)

    finally:
      if session is not None: session.close()

    return result

## Csatlakozás tesztelése

Movies adatbázis letöltése és importálása

In [4]:
# Kapcsoldás az instance-hez
connection = Neo4jConnection(
    "neo4j+s://7434ddc8.databases.neo4j.io",
    "neo4j",
    "7AbyO-0c0otDDCvC9_CLYFWs_fTOd8fNsjj24RnuvTg",
)

In [5]:
# Movies adatbázis letöltése: https://raw.githubusercontent.com/DominikSebe/Adatbazis_projekt2/main/movies.cypher
!wget https://raw.githubusercontent.com/DominikSebe/Adatbazis_projekt2/main/movies.cypher -O movies.cypher

--2024-05-05 11:41:02--  https://raw.githubusercontent.com/DominikSebe/Adatbazis_projekt2/main/movies.cypher
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40596 (40K) [text/plain]
Saving to: ‘movies.cypher’

movies.cypher       100%[===================>]  39.64K  --.-KB/s    in 0.003s  

2024-05-05 11:41:02 (11.6 MB/s) - ‘movies.cypher’ saved [40596/40596]



In [ ]:
# Movies adatbázis létrehozása
with open('/content/movies.cypher', 'r') as cypher:
  for query in filter(lambda s: s.strip() != '', cypher.readlines()):
    connection.run(query.strip())